In [ ]:
import cv2  # Import OpenCV library for image processing
from cvzone.HandTrackingModule import HandDetector  # Import HandDetector from cvzone library for hand tracking
from cvzone.ClassificationModule import Classifier  # Import Classifier from cvzone library for gesture classification
import numpy as np  # Import NumPy library for numerical operations
import math  # Import math library for mathematical functions

In [ ]:
# Initialize video capture from default webcam and create HandDetector object with maxHands set to 2
cap = cv2.VideoCapture(0)
detector = HandDetector(maxHands=2)

In [ ]:
# Load pre-trained model and labels for gesture classification
classifier = Classifier("Model/keras_model.h5", "Model/labels.txt")

# Define variables for cropping and saving images
offset = 20  # Offset for cropping hand regions
imgSize = 300  # Desired size for cropped images
folder = "Data"  # Folder path to save images
labels = ["like", "dislike", "Fist", "one", "two up", "three", "four", "rock", "peace", "stop", "call"]  # List of labels

In [ ]:
# Main loop to capture and process webcam feed
while True:
    # Read a frame from the webcam feed
    success, img = cap.read()
    imgOutput = img.copy()  # Create a copy of the original frame for drawing results
    
    # Detect hands in the frame
    hands, img = detector.findHands(img)
    
    # If hands are detected, process them
    if hands:
        hand = hands[0]  # Get information about the first detected hand
        x, y, w, h = hand['bbox']  # Extract bounding box coordinates
        imgWhite = np.ones((imgSize, imgSize, 3), np.uint8) * 255  # Create white background image
        imgCrop = img[y - offset:y + h + offset, x - offset:x + w + offset]  # Crop hand region from the frame
        
        # If cropped image is not empty, resize and classify it
        if imgCrop.size != 0:
            prediction, index = classifier.getPrediction(imgWhite, draw=False)  # Perform gesture classification
            
            # Draw results on the output frame
            cv2.rectangle(imgOutput, (x - offset, y - offset - 50), (x - offset + 90, y - offset - 50 + 50), (255, 0, 255), cv2.FILLED)
            cv2.putText(imgOutput, labels[index], (x, y - 26), cv2.FONT_HERSHEY_COMPLEX, 1.7, (255, 255, 255), 2)
            cv2.rectangle(imgOutput, (x - offset, y - offset), (x + w + offset, y + h + offset), (255, 0, 255), 4)

In [ ]:
# Display the output frame with drawn results
    cv2.imshow("Image", imgOutput)
    
    # Break the loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video capture and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()
